In [316]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
mm = MinMaxScaler()
ss = StandardScaler()



In [317]:
#등락으로 분류를 시켰기 때문에 등락 퍼센트는 제거해 주었습니다.다른 변수와 상관성이 너무 높은 속성들은 제거처리 해주었습니다.
df=pd.read_csv(r'C:\DRworks\hynix_short.csv',encoding='cp949', index_col=0)
df=df.drop('등락분류',axis=1)
df=df.drop('등락률',axis=1)
df=df.drop('대비',axis=1)
df=df.drop('등락지표',axis=1)
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2670 entries, 2010-07-19 to 2021-05-14
Data columns (total 37 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   등락         2670 non-null   int64  
 1   종가         2670 non-null   float64
 2   시가         2670 non-null   float64
 3   고가         2670 non-null   int64  
 4   저가         2670 non-null   int64  
 5   거래량        2670 non-null   int64  
 6   거래대금       2670 non-null   float64
 7   시가총액       2670 non-null   float64
 8   상장주식수      2670 non-null   int64  
 9   환율         2670 non-null   float64
 10  유가종가       2670 non-null   float64
 11  유가오픈       2670 non-null   float64
 12  유가고가       2670 non-null   float64
 13  유가저가       2670 non-null   float64
 14  유가변동률      2670 non-null   float64
 15  한국은행기준금리   2670 non-null   float64
 16  연준기준금리     2670 non-null   float64
 17  한미기준금리차이   2670 non-null   float64
 18  반도체수출금액지수  2661 non-null   float64
 19  삼성종가       2670 non-null   float64
 20

In [318]:
#xbbost를 위해 x y 로 데이터 분리 후 dmatrix로 변환 (y=등락정도)
X, y = df.iloc[:,1:],df.iloc[:,0]
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2021)
#시기로 분할할 때 데이터
#split = "2012-03-19"
#train = df[:split]
#test= df[split:]

#y_train = train.loc[:,'등락.분류']
#X_train = train.drop('등락.분류', axis=1)
#y_test = test.loc[:,'등락.분류']
#X_test = test.drop('등락.분류', axis=1)


In [319]:

#시기로 분할할 때 데이터
#split = "2012-03-19"
#train = df[:split]
#test= df[split:]

#y_train = train.loc[:,'등락.분류']
#X_train = train.drop('등락.분류', axis=1)
#y_test = test.loc[:,'등락.분류']
#X_test = test.drop('등락.분류', axis=1)

# XGBoost 분류기 생성
xgb_clf = xgb.XGBClassifier()

#초모수 격자생성
xgb_param_grid = {   
                    'colsample_bylevel':[0.9], 
                    'colsample_bytree':[0.8], 
                    'gamma':[0,1,2], 
                    'max_depth':[6,7,8,9], 
                    'min_child_weight':[3,4,5], 
                    'n_estimators':[50,100,150,200], 
                    'nthread':[3,4,5,6], 
                     
                    'random_state':[50,100,150,200] 
                    }
# GridSearchCV 객체 생성
hr_grid = GridSearchCV(estimator=xgb_clf,
                       param_grid=xgb_param_grid,
                       scoring='roc_auc',
                       n_jobs=8,
                       cv=5,
                       refit=True, 
                       return_train_score=True)
hr_grid.fit(X_train, y_train)
#model=XGBClassifier(booster='gbtree', 
                   # colsample_bylevel=0.9, 
                   # colsample_bytree=0.8, 
                   # gamma=0, 
                   # max_depth=8, 
                   # min_child_weight=4, 
                   # n_estimators=100, 
                   # nthread=4, 
                   # objective='multi:softprob', 
                   # random_state=150, 
                   # silent= True)

#model.fit(X_train, y_train,verbose=False)

KeyboardInterrupt: 

In [ ]:
hr_grid_df = pd.DataFrame(hr_grid.cv_results_)
hr_grid_df.loc[:, ['mean_test_score', "params"]]

## 최고성능
best_score = hr_grid.best_score_
# 최고성능을 내는 행을 찾아냄
best_row = hr_grid.best_index_

# 최적 초모수: max_depth, subsample
best_max_depth     = hr_grid.best_params_["max_depth"]
best_max_subsample = hr_grid.best_params_["subsample"]

nl = '\n'
print(f'예측모형성능(AUC):  \t {best_score:.3f}{nl}\
        인덱스:           \t {best_row}{nl}\
        max_depth:      \t {max_depth}{nl}\
        subsample:      \t {subsample}')

In [ ]:
#변수 중요도 그래프
plt.rc('font',family='Malgun Gothic')
plt.rcParams["figure.figsize"] = (10,7)
plot_importance(hr_grid, height=0.9)

In [ ]:
#모델 예측
#y_pred = model.predict(X_test)
#y_true = y_test

#acc = accuracy_score(y_true, y_pred)
#con_mat = confusion_matrix(y_true, y_pred)
#con_mat
#report = classification_report(y_true, y_pred)
#print(report)